# Train Football Player Detector

### Before you start
Use `nvidia-smi` command to see your GPU specs

In [ ]:
!nvidia-smi

Wed Nov  6 17:03:01 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.94                 Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650 ...  WDDM  |   00000000:2B:00.0  On |                  N/A |
| 26%   29C    P8             10W /  110W |     739MiB /   4096MiB |     14%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Make sure which directory you are working on

In [2]:
import os
HOME = os.getcwd()
print(HOME)

c:\Users\alexa\OneDrive\Escritorio\AI Agency Project\ComputerVision\FootballYOLO


### Install dependencies

In [ ]:
%pip install -q ultralytics roboflow

### Imports

In [ ]:
from roboflow import Roboflow
from dotenv import find_dotenv, load_dotenv
from IPython.display import Image

### Configure API keys

In [ ]:
# find .env automatically by walking up directories until it's found
dotenv_path = find_dotenv()

# load up the entries as environment variables
load_dotenv(dotenv_path)

### Pull dataset

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

This dataset is already prepared for model training

In [ ]:
ROBOFLOW_API_KEY = os.getenv("ROBOFLOW_API_KEY")
rf = Roboflow(api_key=ROBOFLOW_API_KEY)

project = rf.workspace("roboflow-jvuqo").project("football-players-detection-3zvbc")
version = project.version(12)
dataset = version.download("yolov8")

Update `data.yaml` structure to align with the latest requirements of the ultralytics package

In [ ]:
!sed -i 's|\(train: \).*|\1../train/images|' {dataset.location}/data.yaml
!sed -i 's|\(val: \).*|\1../valid/images|' {dataset.location}/data.yaml

### Custom Training

Using big image size to detect the ball consistently. 24 GiB of GPU memory is needed to run this training. We can download a pre-trained model if we don't have access to this hardware.

In [ ]:
%cd {HOME}

!yolo task=detect mode=train model=yolov8x.pt data={dataset.location}/data.yaml batch=6 epochs=50 imgsz=1280 plots=True

### Validate custom model
Benchmark the model to verify everything went well, we will use the maP metric to measure the average precission of our model.

In [ ]:
%cd {HOME}

!yolo task=detect mode=val model={HOME}/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml imgsz=1280